# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import logging.config
from logtail import LogtailHandler, LogtailContext
import sys

In [ ]:
#| export

def setup_logging(env, host, token):
    root_logger = logging.getLogger()
    root_logger.setLevel(logging.INFO)
    
    # Clear existing handlers to avoid duplicates if called multiple times
    root_logger.handlers = []
    
    # Add Logtail handler for cloud logging
    logtail_handler = LogtailHandler(
        source_token=token, 
        host=host,
        context=LogtailContext().context(
            environment={'name': env}
        )
    )
    root_logger.addHandler(logtail_handler)
    

    console_handler = logging.StreamHandler(sys.stdout)
    console_formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    console_handler.setFormatter(console_formatter)
    root_logger.addHandler(console_handler)

In [ ]:
#| export 
# If this file is run directly, test the logging
if __name__ == '__main__':
    setup_logging()
    logger = logging.getLogger(__name__)
    logger.info("Test log message")
    logger.error("Test error message") 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()